<a href="https://colab.research.google.com/github/iwanmota-lyft/miscellaneous/blob/main/Test_Contributors_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def filter_contributors(df, include_contributors):
    """
    Filters the DataFrame based on a list of contributors to include.

    Args:
        df (pd.DataFrame): The input DataFrame (e.g., contributorDf).
        include_contributors (list): A list of strings representing the contributors to include.

    Returns:
        pd.DataFrame: A new DataFrame containing only the rows for the specified contributors.
    """
    return df[df['Contributor'].isin(include_contributors)]

print("Function 'filter_contributors' defined successfully.")

# Env
Set environment variables

In [ ]:
import datetime

documentName = 'Risk Tech Project Tracker - Experiment'
# documentName = 'Risk Tech Project Tracker'
sourceSheetName = '2026 Gantt'
dataRange = 'A:I'
contributorsList = ['Arundhati Navada',
                    'Billy Su',
                    'Daniel Briones',
                    'Haomin Shi',
                    'Joon Lee',
                    'Kyle Jang',
                    'Maksim Zhuk',
                    'Masroor Ahmed',
                    'Oleksandr Reshetar',
                    'Sharun Garg',
                    'Tom McNeely'
                    ]

today_date = datetime.date.today().strftime('%Y-%m-%d')
contributorSheetName = f"{today_date}-Contributors"

# Task
Extract data from the given sheet into a dataframe

In [ ]:
import gspread
from google.colab import auth
import google.auth

# Authenticate gspread using user credentials
auth.authenticate_user()

# Obtain the default authenticated credentials
creds, project = google.auth.default()

# Create a gspread client
client = gspread.authorize(creds)

# Open the spreadsheet by name, removing the .gsheet extension if present
spreadsheet = client.open(documentName)
print("Successfully authenticated gspread and opened the spreadsheet.")

Extract the data from the sheet and save to a temp dataframe. This will need to be adapted depending on the input sheet that will be used in the end

In [ ]:
import pandas as pd
# import gspread

# Find the index of the worksheet by name
worksheet_list = spreadsheet.worksheets()
worksheet_index = -1
for i, ws in enumerate(worksheet_list):
    if ws.title == sourceSheetName:
        worksheet_index = i
        break

if worksheet_index != -1:
    print(f"Worksheet '{sourceSheetName}' found at index: {worksheet_index}")
    worksheet = spreadsheet.get_worksheet(worksheet_index)
else:
    print(f"Error: Worksheet '{sourceSheetName}' not found.")
    raise gspread.exceptions.WorksheetNotFound(f"Worksheet '{sourceSheetName}' not found.")

# Get values from range of columns
data = worksheet.get(dataRange)

# The first row of 'data' contains the headers from the sheet.
sheet_headers = data[0]
# The rest of the rows contain the actual data.
sheet_rows = data[4:]

# Create a Pandas DataFrame using the sheet's original headers as temporary column names.
df = pd.DataFrame(sheet_rows, columns=sheet_headers)

# Display the head of the DataFrame
print('DataFrame created successfully')
# print(df.head(15))

Extract/strip/sanitize the contributors into their own rows

In [ ]:
# import pandas as pd
import re

# Create a new temporary dataframe
temp_contributor_df = pd.DataFrame()

# Apply regex-based name splitting - This is the key functionality to split on new line, spaces and commas
temp_contributor_df['Contributor'] = df['Driver'].astype(str).str.replace('\n', ' ').str.findall(r'[A-Z][a-z]+\s+[A-Z][a-z]+')

# Copy 'Launch Date' column from df
temp_contributor_df['Launch Date'] = df['Launch Date']

# Use 'Description' column from df for 'Project' (assuming 'Description' serves as project name/identifier)
# Note: If 'Project' refers to a different column, please specify.
temp_contributor_df['Project'] = df.iloc[:, 1]

print("New temporary DataFrame 'temp_contributor_df' created:")
print(temp_contributor_df.head(20))

## Explode DataFrame

### Subtask:
Use `explode()` to create a new row for each individual contributor, repeating the 'Launch Date' and 'Project' for each.


In [ ]:
contributorDf = temp_contributor_df.explode('Contributor')

print("DataFrame after exploding 'Contributor' column. First 10 rows:")
print(contributorDf.head(10))

Generate a new sheet. Right now the code checks for 1 sheet and deletes/recreates it, but for historical reasons it might be good to generate a new sheet with a timestamp each time this is run

In [ ]:
import gspread
import numpy as np

# Define the name for the new worksheet
# new_worksheet_name = 'Contributor Assignments'

# Check if the worksheet already exists and delete it if it does
try:
    existing_worksheet = spreadsheet.worksheet(contributorSheetName)
    print(f"Worksheet '{contributorSheetName}' already exists. Deleting it...")
    spreadsheet.del_worksheet(existing_worksheet)
    print(f"Successfully deleted existing worksheet: '{contributorSheetName}'")
except gspread.exceptions.WorksheetNotFound:
    print(f"Worksheet '{contributorSheetName}' does not exist. Proceeding to create a new one.")

# Create a new worksheet
new_worksheet = spreadsheet.add_worksheet(title=contributorSheetName, rows=df.shape[0]+1, cols=df.shape[1])
print(f"Successfully created new worksheet: '{contributorSheetName}'")

# Sort the DataFrame before writing
df_sorted = contributorDf.sort_values(by=['Contributor', 'Launch Date']).reset_index(drop=True)

# Replace NaN values with empty strings before converting to list of lists
df_sorted_cleaned = df_sorted.fillna('')

# Filter the dataframe to only include the engineers in Road Safety or Telematics
df_sorted_cleaned_filtered = filter_contributors(df_sorted_cleaned, contributorsList)

# Filter the dataframe to exclude empty description projects
df_sorted_cleaned_filtered = df_sorted_cleaned_filtered[df_sorted_cleaned_filtered['Project'] != '']

# Convert sorted DataFrame to a list of lists, including the header
# This format is required by gspread's update() method
data_to_write = [df_sorted_cleaned_filtered.columns.values.tolist()] + df_sorted_cleaned_filtered.values.tolist()

# Write the data to the new worksheet
new_worksheet.update(data_to_write)

print(f"DataFrame successfully written to '{contributorSheetName}' in the Google Sheet.")